In [ ]:
import numpy
import matplotlib
import lxml
import pandas
import pyteomics
import csv
import math
import multiprocessing
import itertools as it
import xlsxwriter
from pyteomics import mass

In [ ]:
def import_dataframe(file_location):

    if 'csv' in file_location:
        dataframe = pandas.read_csv(file_location)
        dataframe.rename(columns={'m/z':'M(obs)'}, inplace=True)
    else:
        dataframe = pandas.read_excel(file_location)
        dataframe.rename(columns={'m/z':'M(obs)'}, inplace=True)
    return(dataframe)

In [ ]:
def import_obs_masses(dataframe):

    return(list(dataframe['M(obs)']))

In [ ]:
def fragments_multi(prot_seq, obs_mass, cal_type, dataframe, tolerance):

    if cal_type == 'mono':
        aa_comp = dict(mass.std_aa_mass)
        ave_cal = False
    else:
        aa_comp = dict(mass.std_aa_comp)
        ave_cal = True
    
    found = []
    start = 0
    s = int(obs_mass)//107
    e = int(obs_mass)//95
    for frag in prot_seq:
        for i in range(s, e):
            if i > len(prot_seq):
                break
            if math.isclose(round(mass.calculate_mass(prot_seq[start:i], average = ave_cal, aa_comp = aa_comp), 1), obs_mass, abs_tol = tolerance):
                if i == len(prot_seq):
                    find = ['Single',
                            prot_seq[start],
                            int(start + 1),
                            prot_seq[i-1],
                            int(i),
                            obs_mass,
                            round(mass.calculate_mass(prot_seq[start:i], average = ave_cal, aa_comp = aa_comp), 1),
                            round(obs_mass - round(mass.calculate_mass(prot_seq[start:i], average = ave_cal, aa_comp = aa_comp), 1), 1)]
                    found.append(find)
                else:
                    find = ['Double',
                            prot_seq[start],
                            int(start + 1),
                            prot_seq[i-1],
                            int(i),
                            obs_mass,
                            round(mass.calculate_mass(prot_seq[start:i], average = ave_cal, aa_comp = aa_comp), 1),
                            round(obs_mass - round(mass.calculate_mass(prot_seq[start:i], average = ave_cal, aa_comp = aa_comp), 1), 1)]
                    found.append(find)
        s += 1
        e += 1
        start += 1

    return(found)

In [ ]:
# Import the widgets
from ipywidgets import widgets, interact, interactive

# Import the display function for explicitly displaying widgets in the notebook
from IPython.display import display
import ipyupload

#To open files
from ipyupload import FileUpload

style = {'description_width': 'initial'}

seq = widgets.Textarea(value='Input Amino Acid Sequence Here',
                        style = style,
                      layout = widgets.Layout(width='60%'))

file_loc =  widgets.Text(value='Copy and Paste your File Directory Here for you input .xls or .csv observed masses file',
                        layout = widgets.Layout(width='60%', height='25px', justify_content = 'flex-start'),
                        style = style)

mass_cal = widgets.ToggleButtons(options = ['Monoisotopic', 'Average'],
                                 #layout = widgets.Layout(width='99%', height='25px', justify_content = 'flex-start'),
                                 description = 'Mass Calculation Type:',
                                 layout = widgets.Layout(width='60%', height='25px', justify_content = 'flex-start'),
                                 style = style,
                                 button_style='info')
mass_tol = widgets.FloatSlider(value = 0.5, 
                               min=0, 
                               max=2,
                               step=0.1, 
                               style=style,
                               layout = widgets.Layout(width='60%', height='25px', justify_content = 'flex-start'),
                               description='Mass Tolerance (kDa)')

cores = widgets.Dropdown(options=[2, 4, 8, 12, 24],
                          value=2,
                          description = 'Number of Computer Cores:', 
                          style=style, 
                          layout = widgets.Layout(width='60%', height='25px', justify_content = 'flex-start'),
                          readout=True)

#file_upload = FileUpload(
    # https://developer.mozilla.org/en-US/docs/Web/HTML/Element/input#attr-accept
    # eg. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#    accept='', # default
    # True to accept multiple files upload else False
#    multiple=False, # default
    # True to disable the button else False to enable it
#    disabled=False, # default
    # CSS transparently passed to button (a button element overlays the input[type=file] element for better styling)
    # e.g. 'color: darkblue; background-color: lightsalmon; width: 180px;'
#    style_button='color:darkblue; background-color:lightsalmon')

compute = widgets.Button(value=False, 
                         description = 'Find Fragments', 
                         layout = widgets.Layout(width='60%', height='45px', justify_content = 'center'), 
                         disabled = False, 
                         style = {'font_weight': 'bold', 'font-size' : '30px'}, 
                         button_style = 'info', # 'success', 'info', 'warning', 'danger' or '' 
                         tooltip = 'Description', 
                         icon = 'check')
    
display(seq, file_loc, mass_cal, mass_tol, cores, compute)

In [ ]:
cores.value

In [ ]:
pwd